# Review Summarizer
SUMY:  Lex Rank (google), Luhn (wasn't very good), LSA (PCA), Text Rank (google)

## Import

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import dask.dataframe as dd
import pandas as pd
import numpy as np
import math
import os
import pickle
import timeit
from collections import Counter

# Graphics
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='svg'
import seaborn as sns
sns.set(font_scale=1.2)
plt.style.use('seaborn')

In [2]:
from __future__ import print_function
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from operator import itemgetter
import contractions
from langdetect import detect

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Plain text parsers since we are parsing through text
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [4]:
os.chdir('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/notebooks')
import helper_functions
from helper_functions import display_topics

In [54]:
# with open('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/data/reviews_cleaned3.pkl', 'rb') as file:
#     reviews = pickle.load(file)
reviews=pd.read_csv('/Users/bellepeng/Desktop/Metis/Projects/Project_AirBNB/data/SF/201808/reviews_201808.csv')
reviews.columns
drop_list = ['reviewer_id', 'reviewer_name']
print(reviews.shape)
[reviews.drop(x, inplace=True, axis=1) for x in drop_list]
reviews.head(2)

(278884, 6)


,listing_id,id,date,comments
0,958,5977,2009-07-23,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,Returning to San Francisco is a rejuvenating t...


## Pre-process
1. Remove foreign languages, but keep listing_ID and everything else.  
2. Get all the unique IDs of the listings.
3. For each listing:
    - Get the reviews of that one listing
    - Append all the reviews together
    - Break the reviews into sentences  
    - Run the summarizer
    - Save the summary as one row, 3 variables

In [50]:
# running on AWS
reviews_noForeignLang=[]
for comment in reviews_orig['comments']: 
    if not comment:
        reviews_noForeignLang.append(" ")
    try: 
        if detect(comment) == 'en':
            reviews_noForeignLang.append(comment)
        else:
            reviews_noForeignLang.append(" ")
    except:
        reviews_noForeignLang.append(" ")

print(len(reviews_noForeignLang))
print(reviews_orig['comments'][95:103])
print(reviews_noForeignLang[95:103])

8
95                       Great location and great host. 
96     We, a family of four with two little kids, sta...
97                                  非常温馨的家，房东十分友好，小花园安静。
98     Great place, very well located! Well furnished...
99     My husband and I had a wonderful stay at Holly...
100    Une super jolie maison dans le plus beau quart...
101    Very comfortable apartment with loads of priva...
102    Holly was a terrific host - quite attentive an...
Name: comments, dtype: object
['Great location and great host. ', "We, a family of four with two little kids, stayed seven nights at Holly's place. Check-in went very smoothly, Holly helped us unload our car and gave us tips how to avoid a parking fine :-) The flat is very modern and the location is excellent, just a few minutes walk to grocery stores, the tram station, and a great playground. A big plus was that lots of toys were provided, and our kids could play ball in the garage. We would definitely go back.", ' ', 'Great place, ver

In [46]:
test = reviews_orig.iloc[:10]
print(test.shape)
print(test[:2])
test_id = test[['listing_id', 'date']]
test_comments = list(test['comments'])
print(type(test_id), type(test_comments))
# test_combined = pd.DataFrame(np.column_stack([test_id, test_comments]), columns=['id', 'comments'])
test_combined = pd.merge(test_id,pd.DataFrame(test_comments, columns=['comments']),left_index=True, right_index=True)
test_combined

(10, 4)
   listing_id    id        date  \
0         958  5977  2009-07-23   
1         958  6660  2009-08-03   

                                            comments  
0  Our experience was, without a doubt, a five st...  
1  Returning to San Francisco is a rejuvenating t...  
<class 'pandas.core.frame.DataFrame'> <class 'list'>


,listing_id,date,comments
0,958,2009-07-23,"Our experience was, without a doubt, a five st..."
1,958,2009-08-03,Returning to San Francisco is a rejuvenating t...
2,958,2009-09-27,We were very pleased with the accommodations a...
3,958,2009-11-05,We highly recommend this accomodation and agre...
4,958,2010-02-13,Holly's place was great. It was exactly what I...
5,958,2010-03-13,"On top of all that has been said previously, I..."
6,958,2010-04-03,This apartment was everything I could have hop...
7,958,2011-04-08,Great place to stay. I would highly recommend...
8,958,2011-09-26,"Great location in San Francisco, really felt l..."
9,958,2012-07-24,We had a wonderful time at Holly & Davids plac...


Assuming all the comments came back with listing ID and date:  
1. Get all the unique IDs of the listings.
2. For each listing:
    - Get the reviews of that one listing
    - Append all the reviews together
    - Break the reviews into sentences
    - Run the summarizer
    - Save the summary as one row, 3 variables

In [12]:
# Get all the IDs of the listings
listing_ids=reviews['listing_id'].unique()
print(len(listing_ids))
listing_ids[:5]

5491


array([ 958, 5858, 7918, 8142, 8339])

## Model

In [141]:
def review_summarizer(text, num_sentences=3, printToScreen=False):
    parser = PlaintextParser(text, Tokenizer('english'))
    
    Summarizer_LexRank = LexRankSummarizer()
    summary_LexRank = Summarizer_LexRank(parser.document, num_sentences) 
    if printToScreen==True:
        print("LexRank Summarizer:")
        for sentence in summary_LexRank:
            print(sentence)
        print()
    
    Summarizer_Lsa = LsaSummarizer()
    summary_LSA = Summarizer_Lsa(parser.document, num_sentences)
    if printToScreen==True:
        print("LSA Summarizer:")
        for sentence in summary_LSA:
            print(sentence)
        print()
    
    Summarizer_Luhn = LuhnSummarizer()
    summary_Luhn = Summarizer_Luhn(parser.document, num_sentences)
    if printToScreen==True:
        print("Luhn Summarizer:")
        for sentence in summary_Luhn:
            print(sentence)
        print()
    
    Summarizer_TextRank = TextRankSummarizer()
    summary_TextRank = Summarizer_TextRank(parser.document, num_sentences)
    if printToScreen==True:
        print("Text Rank Summarizer:")
        for sentence in summary_TextRank:
            print(sentence)
        print()
    
    return  summary_LexRank, summary_LSA, summary_Luhn, summary_TextRank
    #Summarizer_LexRank, Summarizer_Lsa, Summarizer_Luhn, Summarizer_TextRank, \

In [28]:
summary_LexRank, summary_LSA, summary_Luhn, summary_TextRank = review_summarizer(reviews_one_listing2, num_sentences=2)

LexRank Summarizer:
Holly's apartment was a great stay!
The location was great, lovely part of the city and a great base to explore.. We were at Holly's place for 10 days, and we enjoy it so much!!!

LSA Summarizer:
Located near public transport and a very affordable Lyft ride from most places in the city, I would love to stay at Holly's again.
The bed is very comfortable.. 무엇보다 위치가 제일 마음에 들었습니다 공항과 시내를 왔다갔다 하기좋았고 늦게들어가도 밤에 무섭지 않아서 좋았습니다 마트가 가까이 있어서 너무 편리했어요!

Luhn Summarizer:
We would happily stay there again and would highly recommend the place to others.. Una parte muy importante de cada viaje es el lugar donde uno se hospedará durante la estadía en el extranjero, con toda seguridad puedo decir que el apartamento de Holly Coleman nos brindó una excelente experiencia, ya que la localización de éste es ideal para alguien que desea conocer San Francisco con calma y teniendo los barrios más interesantes en una distancia muy corta.
Una parte muy importante de cada viaje es el lugar donde

In [150]:
# Implement for all
def summarize_all_reviews(reviews, listing_ids, num_sentences=3, printToScreen=False):
    summaryLexRank = []
    summaryLSA = []
    summaryLuhn = []
    summaryTextRank = []
    
    # Get the reviews for one listing at a time
    for i in listing_ids:
        ix_one_review = list(reviews[reviews['listing_id']==i].index)
        reviews_one_listing = [reviews['comments'].iloc[ix] for ix in ix_one_review]
    
        # Prepare the data: put all the reviews together
        reviews_one_listing2 = (" ").join(item for item in reviews_one_listing if isinstance(item, str))
        
        # Summarize
        sum1_LexRank, sum1_LSA, sum1_Luhn, sum1_TextRank = \
        review_summarizer(reviews_one_listing2, num_sentences=num_sentences, printToScreen=printToScreen)
        
        # Add to list of summaries
        summaryLexRank.append(str(sum1_LexRank[0])+" "+str(sum1_LexRank[1]))
        summaryLSA.append(str(sum1_LSA[0])+" "+str(sum1_LSA[1]))
        summaryLuhn.append(str(sum1_Luhn[0])+" "+str(sum1_Luhn[1]))
        summaryTextRank.append(str(sum1_TextRank[0])+" "+str(sum1_TextRank[1]))
    
    
    return summaryLexRank, summaryLSA, summaryLuhn, summaryTextRank

In [151]:
summaryLexRank, summaryLSA, summaryLuhn, summaryTextRank = \
    summarize_all_reviews(reviews, listing_ids[:2], num_sentences=2, printToScreen=False)

In [152]:
print(summaryLexRank)
print()
print(summaryLSA)
print()
print(summaryTextRank)
print()
print(summaryLuhn)

["Holly's apartment was the perfect place for us to stay on our trip to SF! Holly's apartment was a great stay!", "We had a lovely stay at Philip and Tania's home in a great location in SF. Tania and Philip are very nice hosts and the house is wonderful."]

["You do not need a car to enjoy the City as everything is within walking distance - great restaurants, bars and local stores. Located near public transport and a very affordable Lyft ride from most places in the city, I would love to stay at Holly's again.", 'There is also a small cafe right down the street where you can get coffee, breakfast, and check your email free of charge. As we did not really know what to do in San Francisco, except the touristic stuff, Philip planed each day with us although he was ill.']

["The apartment was a charming layout with a full view and access to the home's garden The location is perfect for full engagement with the city; close to mass transit with walking proximity to the Haight, the Mission, t